## Optimization in Electricity Markets

This notebook will closely follow examples described and given in Optimization Models in Electricity Markets by Anthony Papavasiliou. The purpose of this notebook is to gain experience and show how to use industry examples of this. The framework chosen for this is pyomo since it is the most well known free framework that can be used by anybody. It isn't ideal for matrices or vector constraints so they are not natively supported but still useful. 

In [1]:
# Import the necessary libraries
import numpy as np
import polars as pl
import jax.numpy as jnp
import pyomo.environ as pyo

### Example 2.3: 

Consider the following Example where we are focussed on three generators with the goal of producing a demand of 200 MW at a minimum cost. 
Each generator can either be turned on or off. Let $p_i$ denote the power produced by generator $i$ and $\mu_i$ be a binary variable that indicates whether a generator has been activated or not.

\begin{array}{lccc}
\hline \text { Generator } & \text { Activation cost (\$) } & \text { Marg. cost (\$/MWh) } & \text { Capacity (MW) } \\
\hline \text { Cheap } & 500 & 0 & 20 \\
\text { Moderate } & 1000 & 10 & 100 \\
\text { Expensive } & 2000 & 80 & 100 \\
\hline
\end{array}


The minimization problem can be expressed as 

\[
\begin{aligned}
& \min_{p,u}\; 500u_1 + 1000u_2 + 10p_2 + 2000u_3 + 80p_3 \\
& \text{s.t.}\; 0 \le p_1 \le 20u_1 \\
& \phantom{\text{s.t.}}\; 0 \le p_2 \le 100u_2 \\
& \phantom{\text{s.t.}}\; 0 \le p_3 \le 100u_3 \\
& \lambda:\; p_1 + p_2 + p_3 = 200 \\
& u_i \in \{0,1\}
\end{aligned}
\]

In [2]:
# Create or instantiate the model object
model = pyo.ConcreteModel()

# define the model types: Binary variables
model.u1 = pyo.Var(domain=pyo.Binary)
model.u2 = pyo.Var(domain=pyo.Binary)
model.u3 = pyo.Var(domain=pyo.Binary)

# define the model types: nonnegative real numbers
model.p1 = pyo.Var(domain = pyo.NonNegativeReals)
model.p2 = pyo.Var(domain = pyo.NonNegativeReals)
model.p3 = pyo.Var(domain = pyo.NonNegativeReals)

# define the objective function
model.obj = pyo.Objective(expr = 500 * model.u1 +1000 * model.u2 + 10* model.p2 + 2000 * model.u3 + 80 * model.p3,
                          sense = pyo.minimize)


# Capacity upper bounds: 0 ≤ p_i ≤ cap_i·u_i (lower bound already enforced by NonNegativeReals)
model.capacity_u1 = pyo.Constraint(expr=model.p1 <= 20 * model.u1)
model.capacity_u2 = pyo.Constraint(expr=model.p2 <= 100 * model.u2)
model.capacity_u3 = pyo.Constraint(expr=model.p3 <= 100 * model.u3)

# Demand balance: p1 + p2 + p3 = 200 MW
model.demand_balance = pyo.Constraint(expr=model.p1 + model.p2 + model.p3 == 200)


This has setup the basic of the model so far but know we need to decide how we want to solve this and pyomo is a framework that can work with differnt solver using the SolverFactory arguement. Lets check which solvers are available for pyomo using the following command in the termninal: pyomo help --solvers 

This gives the complete list of solvers. Now we need to set this based on what we have installed. Note that MOSEK and CPLEX need to be installed separately so we won't use them for now. But very useful 

In [3]:
# list of differnet solvers 
candidates = ["glpk","highs","cbc","cplex","ipopt","mindtpy","mosek"]
print("Solver availability:\n")
for name in candidates:
    available = pyo.SolverFactory(name).available(False)
    print(f"{name:<7} -> {available}")

Solver availability:

glpk    -> True
highs   -> True
cbc     -> True
cplex   -> False
ipopt   -> True
mindtpy -> True
mosek   -> False


Now we can set the solver. Choosing cbc for this case! But what if we want to set the type of solver that CBC uses? 

In [5]:
solver = pyo.SolverFactory('cbc')
result = solver.solve(model)

print(result)


Problem: 
- Name: unknown
  Lower bound: 10900.0
  Upper bound: 10900.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 3
  Number of nonzeros: 5
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.043439626693725586
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [6]:

print(f"Total Dollars: ${pyo.value(model.obj):,.2f}")
print(f"Generator 1 is set: {model.u1.value:.0f} with power of {model.p1.value:.1f} MW/h")
print(f"Generator 2 is set: {model.u2.value:.0f} with power of {model.p2.value:.1f} MW/h")
print(f"Generator 3 is set: {model.u3.value:.0f} with power of {model.p3.value:.1f} MW/h")


Total Dollars: $10,900.00
Generator 1 is set: 1 with power of 20.0 MW/h
Generator 2 is set: 1 with power of 100.0 MW/h
Generator 3 is set: 1 with power of 80.0 MW/h


We got an ideal result here but in terms of longevity we have issues since what if we want to run this for multiple generators, more constraints, but the same objective function in general. This is why while the above format is technically valid it is more ideal to run this in a more functional form. 